In [1]:
# Below dataset are not provided in the repository because they are confidential

In [ ]:
import pandas as pd
# this dataset contains the meta data of product 
df = pd.read_csv('final_dataset_that_contains_sales_volume.csv')
df

,product_id,description,g:product_type,tag,name,jan,mpn,price,special_price,Category Group,Specifications,combined_text_features,volume_of_sales
0,58641,"Sporting a classic crew neck, this olive tee b...",Tops & T-Shirts &gt; All Products &gt; T-shirts,WAW Under 999,Olive Crew Neck T-shirt,OSM NOOS,T-Shirts,799.0,319.0,T-Shirts | Crew Neck,Green | Cotton 100% | Camo Print | Half Slee...,Olive Crew Neck T-shirt WAW Under 999 Sportin...,1311.0
1,383422,JACK & JONES Black All Over Caution Print Pyjamas,Loungewear &gt; All Products &gt; Pyjamas,12SW Under 999,Black All Over Caution Print Pyjamas,SS21 UG,Pyjamas,1299.0,779.0,Loungewear | Pyjamas,Black | Above Knee | 100% Cotton | Printed | C...,Black All Over Caution Print Pyjamas 12SW Unde...,193.0
2,59108,"Strike a pose in these cool, lace-up sneakers ...",Footwear &gt; All Products &gt; Sneakers,NaN,Black Contrast Detail Sneakers,SS21 FA,Sneakers,5999.0,2999.0,Footwear | Sneakers,"Black | 60% Polyurethane, 40% Upper Mesh | Pla...",Black Contrast Detail Sneakers Strike a pose ...,293.0
3,44149,Take on a comfy look wearing this black drawst...,Athleisure &gt; All Products &gt; Sweatpants,WAW Under 999,Black Drawstring Trackpants,AW19 UGD,Trackpants,1599.0,879.0,Athleisure | Sweatpants,Black | 80% Cotton 20% Polyester | Plain Colou...,Black Drawstring Trackpants WAW Under 999 Take...,126.0
4,44677,Stay at ease all day long by wearing this pair...,Athleisure &gt; All Products &gt; Sweatpants,WAW Under 999,Black Drawstring Trackpants,OSM UG,Trackpants,1699.0,934.0,Athleisure | Sweatpants,Black | 100% Polyester | Plain Coloured | Casu...,Black Drawstring Trackpants WAW Under 999 Stay...,103.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6832,416595,Make a statement in these skinny fit leans fro...,New In &gt; Jeans &gt; Skinny Fit,NaN,Light Blue Low Rise Ben Skinny Fit Jeans,SS24 NOOS,Jeans,2999.0,0.0,Jeans | Skinny Fit | 12271267 | SMU,"Blue | 66% Cotton,33% Polyester,1% Elastane | ...",Light Blue Low Rise Ben Skinny Fit Jeans Make...,NaN
6833,416811,"Introducing our Black Colourblocked Shorts, th...",ALL PRODUCTS &gt; Bottomwear &gt; Shorts,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,"Introducing our Black Colourblocked Shorts, ...",NaN
6834,416812,"Hop into style with our ""Yellow Bugs Bunny Hoo...",New In &gt; Kids New in &gt; T-Shirts,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,"Hop into style with our ""Yellow Bugs Bunny H...",NaN
6835,416813,Hop into style with this Bugs Bunny Print T-sh...,New In &gt; Kids New in &gt; T-Shirts,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Hop into style with this Bugs Bunny Print T-...,NaN


In [ ]:
quarter_sales = pd.read_csv('product_id_with_daily_volume_sales.csv')
quarter_sales.head()

,date,product_id,volme_of_sales
0,2019-07-01,43540,1
1,2019-07-01,43556,1
2,2019-07-01,44130,1
3,2019-07-01,44148,1
4,2019-07-02,44130,2


In [ ]:
# Calculating the new columns
df['discount_price_ratio'] = df['special_price'] / df['price']
df['price_sales_ratio'] = df['price'] / df['volume_of_sales']

In [ ]:
# Convert 'date' to datetime format and extract the quarter
quarter_sales['date'] = pd.to_datetime(quarter_sales['date'])
quarter_sales['quarter'] = quarter_sales['date'].dt.quarter

# Pivot table to summarize sales for each product in each quarter
quarter_sales = quarter_sales.pivot_table(values='volme_of_sales', index='product_id', columns='quarter', aggfunc='sum', fill_value=0)
quarter_sales.columns = ['Quarter1_sales', 'Quarter2_sales', 'Quarter3_sales', 'Quarter4_sales']

# Find the quarter with the highest sales for each product
quarter_sales['max_quarter'] = quarter_sales.idxmax(axis=1)

# Initialize new columns to 0
quarter_sales['Quarter1_sales'] = 0
quarter_sales['Quarter2_sales'] = 0
quarter_sales['Quarter3_sales'] = 0
quarter_sales['Quarter4_sales'] = 0

# Set the appropriate column to 1 based on max_quarter
for i, row in quarter_sales.iterrows():
    quarter_sales.at[i, row['max_quarter']] = 1

# Drop the max_quarter column as it's no longer needed
quarter_sales = quarter_sales.drop(columns=['max_quarter'])

print(quarter_sales)

            Quarter1_sales  Quarter2_sales  Quarter3_sales  Quarter4_sales
product_id                                                                
41514                    0               0               1               0
41899                    0               0               0               1
43537                    0               0               1               0
43540                    0               0               1               0
43556                    0               0               1               0
...                    ...             ...             ...             ...
416589                   0               1               0               0
416591                   0               1               0               0
416592                   0               1               0               0
416593                   0               1               0               0
416594                   0               1               0               0

[6618 rows x 4 columns]


In [ ]:
# Resetting index of quarter_sales to merge on product_id
quarter_sales.reset_index(inplace=True)

# Merging the two DataFrames
merged_df = pd.merge(df, quarter_sales, on='product_id', how='left')

merged_df

,product_id,description,g:product_type,tag,name,jan,mpn,price,special_price,Category Group,Specifications,combined_text_features,volume_of_sales,discount_price_ratio,price_sales_ratio,Quarter1_sales,Quarter2_sales,Quarter3_sales,Quarter4_sales
0,58641,"Sporting a classic crew neck, this olive tee b...",Tops & T-Shirts &gt; All Products &gt; T-shirts,WAW Under 999,Olive Crew Neck T-shirt,OSM NOOS,T-Shirts,799.0,319.0,T-Shirts | Crew Neck,Green | Cotton 100% | Camo Print | Half Slee...,Olive Crew Neck T-shirt WAW Under 999 Sportin...,1311.0,0.399249,0.609458,0.0,0.0,1.0,0.0
1,383422,JACK & JONES Black All Over Caution Print Pyjamas,Loungewear &gt; All Products &gt; Pyjamas,12SW Under 999,Black All Over Caution Print Pyjamas,SS21 UG,Pyjamas,1299.0,779.0,Loungewear | Pyjamas,Black | Above Knee | 100% Cotton | Printed | C...,Black All Over Caution Print Pyjamas 12SW Unde...,193.0,0.599692,6.730570,1.0,0.0,0.0,0.0
2,59108,"Strike a pose in these cool, lace-up sneakers ...",Footwear &gt; All Products &gt; Sneakers,NaN,Black Contrast Detail Sneakers,SS21 FA,Sneakers,5999.0,2999.0,Footwear | Sneakers,"Black | 60% Polyurethane, 40% Upper Mesh | Pla...",Black Contrast Detail Sneakers Strike a pose ...,293.0,0.499917,20.474403,0.0,0.0,0.0,1.0
3,44149,Take on a comfy look wearing this black drawst...,Athleisure &gt; All Products &gt; Sweatpants,WAW Under 999,Black Drawstring Trackpants,AW19 UGD,Trackpants,1599.0,879.0,Athleisure | Sweatpants,Black | 80% Cotton 20% Polyester | Plain Colou...,Black Drawstring Trackpants WAW Under 999 Take...,126.0,0.549719,12.690476,0.0,0.0,0.0,1.0
4,44677,Stay at ease all day long by wearing this pair...,Athleisure &gt; All Products &gt; Sweatpants,WAW Under 999,Black Drawstring Trackpants,OSM UG,Trackpants,1699.0,934.0,Athleisure | Sweatpants,Black | 100% Polyester | Plain Coloured | Casu...,Black Drawstring Trackpants WAW Under 999 Stay...,103.0,0.549735,16.495146,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6832,416595,Make a statement in these skinny fit leans fro...,New In &gt; Jeans &gt; Skinny Fit,NaN,Light Blue Low Rise Ben Skinny Fit Jeans,SS24 NOOS,Jeans,2999.0,0.0,Jeans | Skinny Fit | 12271267 | SMU,"Blue | 66% Cotton,33% Polyester,1% Elastane | ...",Light Blue Low Rise Ben Skinny Fit Jeans Make...,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
6833,416811,"Introducing our Black Colourblocked Shorts, th...",ALL PRODUCTS &gt; Bottomwear &gt; Shorts,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,"Introducing our Black Colourblocked Shorts, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
6834,416812,"Hop into style with our ""Yellow Bugs Bunny Hoo...",New In &gt; Kids New in &gt; T-Shirts,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,"Hop into style with our ""Yellow Bugs Bunny H...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
6835,416813,Hop into style with this Bugs Bunny Print T-sh...,New In &gt; Kids New in &gt; T-Shirts,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Hop into style with this Bugs Bunny Print T-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
merged_df.drop(columns=['tag'],inplace=True)

In [ ]:
merged_df.isna().sum()

product_id                  0
description                 0
g:product_type              0
name                        4
jan                         4
mpn                         4
price                       0
special_price               0
Category Group              6
Specifications              4
combined_text_features      0
volume_of_sales           219
discount_price_ratio        4
price_sales_ratio         219
Quarter1_sales            219
Quarter2_sales            219
Quarter3_sales            219
Quarter4_sales            219
dtype: int64

In [ ]:
merged_df.head()

,product_id,description,g:product_type,name,jan,mpn,price,special_price,Category Group,Specifications,combined_text_features,volume_of_sales,discount_price_ratio,price_sales_ratio,Quarter1_sales,Quarter2_sales,Quarter3_sales,Quarter4_sales
0,58641,"Sporting a classic crew neck, this olive tee b...",Tops & T-Shirts &gt; All Products &gt; T-shirts,Olive Crew Neck T-shirt,OSM NOOS,T-Shirts,799.0,319.0,T-Shirts | Crew Neck,Green | Cotton 100% | Camo Print | Half Slee...,Olive Crew Neck T-shirt WAW Under 999 Sportin...,1311.0,0.399249,0.609458,0.0,0.0,1.0,0.0
1,383422,JACK & JONES Black All Over Caution Print Pyjamas,Loungewear &gt; All Products &gt; Pyjamas,Black All Over Caution Print Pyjamas,SS21 UG,Pyjamas,1299.0,779.0,Loungewear | Pyjamas,Black | Above Knee | 100% Cotton | Printed | C...,Black All Over Caution Print Pyjamas 12SW Unde...,193.0,0.599692,6.730570,1.0,0.0,0.0,0.0
2,59108,"Strike a pose in these cool, lace-up sneakers ...",Footwear &gt; All Products &gt; Sneakers,Black Contrast Detail Sneakers,SS21 FA,Sneakers,5999.0,2999.0,Footwear | Sneakers,"Black | 60% Polyurethane, 40% Upper Mesh | Pla...",Black Contrast Detail Sneakers Strike a pose ...,293.0,0.499917,20.474403,0.0,0.0,0.0,1.0
3,44149,Take on a comfy look wearing this black drawst...,Athleisure &gt; All Products &gt; Sweatpants,Black Drawstring Trackpants,AW19 UGD,Trackpants,1599.0,879.0,Athleisure | Sweatpants,Black | 80% Cotton 20% Polyester | Plain Colou...,Black Drawstring Trackpants WAW Under 999 Take...,126.0,0.549719,12.690476,0.0,0.0,0.0,1.0
4,44677,Stay at ease all day long by wearing this pair...,Athleisure &gt; All Products &gt; Sweatpants,Black Drawstring Trackpants,OSM UG,Trackpants,1699.0,934.0,Athleisure | Sweatpants,Black | 100% Polyester | Plain Coloured | Casu...,Black Drawstring Trackpants WAW Under 999 Stay...,103.0,0.549735,16.495146,0.0,1.0,0.0,0.0


In [2]:
# below code is final logic in which we combine the cosine similary and TF-IDF to make item item recommender system

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack, csr_matrix
from sklearn.decomposition import TruncatedSVD

# Fill missing values with empty strings for text columns and with 0 for numerical columns
merged_df['name'].fillna('', inplace=True)
merged_df['Category Group'].fillna('', inplace=True)
merged_df['description'].fillna('', inplace=True)
merged_df['g:product_type'].fillna('', inplace=True)
merged_df['Specifications'].fillna('', inplace=True)
merged_df['jan'].fillna('', inplace=True)
merged_df['mpn'].fillna('', inplace=True)
merged_df['price'].fillna(0, inplace=True)
merged_df['special_price'].fillna(0, inplace=True)
merged_df['volume_of_sales'].fillna(0, inplace=True)
merged_df['discount_price_ratio'].fillna(0, inplace=True)
merged_df['price_sales_ratio'].fillna(0, inplace=True)
merged_df['Quarter1_sales'].fillna(0, inplace=True)
merged_df['Quarter2_sales'].fillna(0, inplace=True)
merged_df['Quarter3_sales'].fillna(0, inplace=True)
merged_df['Quarter4_sales'].fillna(0, inplace=True)

# Combine relevant text features into a single string for each item
merged_df['combined_text_features'] = merged_df.apply(lambda row: ' '.join([
    str(row['name']),
    str(row['description']),
    str(row['mpn']),
    str(row['g:product_type']),
    str(row['Category Group']),
]), axis=1)

# Vectorize the combined text features using TF-IDF
text_vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)  # Limit to 10,000 features for memory efficiency
tfidf_matrix = text_vectorizer.fit_transform(merged_df['combined_text_features'])

# Reduce dimensionality of TF-IDF matrix
svd = TruncatedSVD(n_components=100)  # Reduce to 100 dimensions
tfidf_matrix_reduced = svd.fit_transform(tfidf_matrix)
tfidf_matrix_reduced = csr_matrix(tfidf_matrix_reduced)  # Convert to sparse matrix

# Compute the cosine similarity matrix for textual features
cosine_sim_matrix_text = cosine_similarity(tfidf_matrix_reduced, tfidf_matrix_reduced)

# Function to get initial recommendations based on text features
def get_initial_recommendations(product_id, cosine_sim_matrix, merged_df, top_n=50):
    # Get the index of the product that matches the product_id
    idx = merged_df.index[merged_df['product_id'] == product_id].tolist()[0]

    # Get the pairwise similarity scores of all products with that product
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))

    # Sort the products based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the top_n most similar products
    sim_scores = sim_scores[1:top_n+1]

    # Get the product indices
    product_indices = [i[0] for i in sim_scores]

    # Return the top_n most similar products
    return merged_df.iloc[product_indices].reset_index(drop=True)

# Function to get final recommendations based on numerical features
def get_final_recommendations(initial_recommendations, top_n=10):
    # Scale the numerical features
    numerical_features = initial_recommendations[['price', 'special_price', 'volume_of_sales','discount_price_ratio','price_sales_ratio']].values

    scaler = MinMaxScaler()
    scaled_numerical_features = scaler.fit_transform(numerical_features)
    scaled_numerical_features = csr_matrix(scaled_numerical_features)  # Convert to sparse matrix

    quarter_sales_features = initial_recommendations[['Quarter1_sales', 'Quarter2_sales', 'Quarter3_sales', 'Quarter4_sales']].values
    quarter_sales_features = csr_matrix(quarter_sales_features)  # Convert to sparse matrix

    combined_features = hstack([scaled_numerical_features, quarter_sales_features])

    # Compute the cosine similarity matrix for numerical features
    cosine_sim_matrix_num = cosine_similarity(combined_features, combined_features)

    # Get the index of the product in the initial recommendations DataFrame
    product_id = initial_recommendations.iloc[0]['product_id']
    idx = initial_recommendations.index[initial_recommendations['product_id'] == product_id].tolist()[0]

    # Get the pairwise similarity scores of all products with that product
    sim_scores = list(enumerate(cosine_sim_matrix_num[idx]))

    # Sort the products based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the top_n most similar products
    sim_scores = sim_scores[1:top_n+1]

    # Get the product indices
    product_indices = [i[0] for i in sim_scores]

    # Return the top_n most similar products
    return initial_recommendations.iloc[product_indices]

# Main function to get recommendations
def get_recommendations(product_id, merged_df, top_n=10):
    initial_recommendations = get_initial_recommendations(product_id, cosine_sim_matrix_text, merged_df, top_n=50)
    final_recommendations = get_final_recommendations(initial_recommendations, top_n)
    return final_recommendations


In [ ]:
product_id = 58641
recommended_products = get_recommendations(product_id, merged_df, top_n=10)
recommended_products[['product_id','name','Category Group','price']]

,product_id,name,Category Group,price
28,397205,White Crew Neck T-shirt,T-Shirts | Crew Neck,999.0
33,397206,Black Crew Neck T-shirt,T-Shirts | Crew Neck,999.0
20,398244,White Crew Neck T-shirt,T-Shirts | Crew Neck,1499.0
35,394911,Peach Crew Neck T-shirt,T-Shirts | Crew Neck,1799.0
11,58223,Red Crew Neck T-shirt,T-Shirts | Crew Neck,799.0
18,410919,Black Logo Branding Crew Neck T-shirt,T-Shirts | Crew Neck | 12261186 | FASHION NOOS,799.0
22,406233,Green Crew Neck T-shirt,T-Shirts | Crew Neck | 12244255 | Mainline,1799.0
49,391647,Green Ctect Print Crew Neck T-shirt,T-Shirts | Crew Neck,2299.0
24,408905,Black Text Print Crew Neck T-shirt,T-Shirts | Crew Neck | 12256559 | FASHION NOOS,2299.0
31,403982,Yellow Self-Design Crew Neck T-shirt,T-Shirts | Crew Neck | 12244336 | Mainline,1799.0


In [ ]:
product_id = 59108
recommended_products = get_recommendations(product_id, merged_df, top_n=10)
recommended_products[['product_id','name','Category Group','price']]

,product_id,name,Category Group,price
1,399107,Black Hi-Top Sneakers,Footwear | Sneakers,4499.0
18,399108,White Colourblocked Hi-Top Sneakers,Footwear | Sneakers,4499.0
46,405308,White &amp; Black Colourblocked Lace-Up Sneakers,Footwear | Sneakers | 12235596 | Fashion,5999.0
12,406523,White Mid-Top Lace-Up Sneakers,Footwear | Sneakers | 12235595 | Fashion,5999.0
38,405314,White Lace-Up Sneakers,Footwear | Sneakers | 12235587 | Fashion,4299.0
42,406742,Black Slip-On Sneakers,Footwear | Sneakers | 12235583 | Fashion,4299.0
6,399838,Green Mesh Detail Sneakers,Footwear | Sneakers,3999.0
22,405316,Blue Sneakers,Footwear | Sneakers | 12235588 | Fashion,4299.0
33,405311,Blue &amp; Black Printed Lace-Up Sneakers,Footwear | Sneakers | 12235586 | Fashion,4299.0
14,401857,BLACK LACE-UP MESH SNEAKERS,Footwear | Sneakers,3499.0


In [ ]:
product_id = 58641
recommended_products = get_recommendations(product_id, merged_df, top_n=10)
recommended_products[['product_id','name','Category Group','price']]

,product_id,name,Category Group,price
28,397205,White Crew Neck T-shirt,T-Shirts | Crew Neck,999.0
33,397206,Black Crew Neck T-shirt,T-Shirts | Crew Neck,999.0
20,398244,White Crew Neck T-shirt,T-Shirts | Crew Neck,1499.0
35,394911,Peach Crew Neck T-shirt,T-Shirts | Crew Neck,1799.0
11,58223,Red Crew Neck T-shirt,T-Shirts | Crew Neck,799.0
18,410919,Black Logo Branding Crew Neck T-shirt,T-Shirts | Crew Neck | 12261186 | FASHION NOOS,799.0
22,406233,Green Crew Neck T-shirt,T-Shirts | Crew Neck | 12244255 | Mainline,1799.0
49,391647,Green Ctect Print Crew Neck T-shirt,T-Shirts | Crew Neck,2299.0
24,408905,Black Text Print Crew Neck T-shirt,T-Shirts | Crew Neck | 12256559 | FASHION NOOS,2299.0
31,403982,Yellow Self-Design Crew Neck T-shirt,T-Shirts | Crew Neck | 12244336 | Mainline,1799.0


In [ ]:
merged_df.loc[merged_df['product_id']==58641][['product_id','name','Category Group','price']]

,product_id,name,Category Group,price
0,58641,Olive Crew Neck T-shirt,T-Shirts | Crew Neck,799.0


In [ ]:
product_id = 44149
recommended_products = get_recommendations(product_id, merged_df, top_n=10)
recommended_products[['product_id','name','Category Group','price']]

,product_id,name,Category Group,price
1,44150,Dark Grey Drawstring Trackpants,Athleisure | Sweatpants,1599.0
48,389321,Blue Mid Rise Logo Print Trackpants,Athleisure | Sweatpants,1299.0
34,385918,Maroon Mid Rise Trackpants,Athleisure | Sweatpants,1799.0
40,385917,Grey Mid Rise Trackpants,Athleisure | Sweatpants,1799.0
35,395463,Grey Trackpants,Athleisure | Sweatpants,1799.0
9,386222,Grey Mid Rise Trackpants,Athleisure | Sweatpants,1699.0
6,59821,Black Textured Trackpants,Athleisure | Sweatpants,2299.0
44,389323,Blue Mid Rise Trackpants,Athleisure | Sweatpants,1699.0
18,59822,Navy Blue Textured Trackpants,Athleisure | Sweatpants,2299.0
15,394284,Black Trackpants,Athleisure | Sweatpants,1999.0


In [ ]:
merged_df.loc[merged_df['product_id']==44149][['product_id','name','Category Group','price']]

,product_id,name,Category Group,price
3,44149,Black Drawstring Trackpants,Athleisure | Sweatpants,1599.0


In [ ]:
product_id = 383422
recommended_products = get_recommendations(product_id, merged_df, top_n=10)
recommended_products[['product_id','name','Category Group','price']]

,product_id,name,Category Group,price
23,401156,Red Printed Pyjamas,Loungewear | Pyjamas,1299.0
27,394852,Blue Printed Pyjamas,Loungewear | Pyjamas,1299.0
12,401158,White Printed Pyjamas,Loungewear | Pyjamas,1299.0
6,401162,Olive Printed Pyjamas,Loungewear | Pyjamas,1299.0
33,394265,Grey Camo Pyjamas,Loungewear | Pyjamas,1199.0
1,401164,Black Printed Pyjamas,Loungewear | Pyjamas,1299.0
22,401163,Dark Blue Printed Pyjamas,Loungewear | Pyjamas,1299.0
24,401161,Red Printed Pyjamas,Loungewear | Pyjamas,1299.0
20,403423,White Printed Cotton Pyjamas,Loungewear | Pyjamas | 12232845 | Fashion,1299.0
17,386245,Yellow Mid Rise Check Pyjamas,Loungewear | Pyjamas,1199.0


In [ ]:
merged_df.loc[merged_df['product_id']==383422][['product_id','name','Category Group','price']]

,product_id,name,Category Group,price
1,383422,Black All Over Caution Print Pyjamas,Loungewear | Pyjamas,1299.0


In [ ]:
product_id = 44677
recommended_products = get_recommendations(product_id, merged_df, top_n=10)
recommended_products[['product_id','name','Category Group','price']]

,product_id,name,Category Group,price
3,44150,Dark Grey Drawstring Trackpants,Athleisure | Sweatpants,1599.0
1,44151,Navy Drawstring Trackpants,Athleisure | Sweatpants,1599.0
43,389321,Blue Mid Rise Logo Print Trackpants,Athleisure | Sweatpants,1299.0
28,385918,Maroon Mid Rise Trackpants,Athleisure | Sweatpants,1799.0
38,385917,Grey Mid Rise Trackpants,Athleisure | Sweatpants,1799.0
27,395463,Grey Trackpants,Athleisure | Sweatpants,1799.0
20,386222,Grey Mid Rise Trackpants,Athleisure | Sweatpants,1699.0
4,59821,Black Textured Trackpants,Athleisure | Sweatpants,2299.0
40,389323,Blue Mid Rise Trackpants,Athleisure | Sweatpants,1699.0
17,59822,Navy Blue Textured Trackpants,Athleisure | Sweatpants,2299.0


In [ ]:
merged_df.loc[merged_df['product_id']==44677][['product_id','name','Category Group','price']]

,product_id,name,Category Group,price
4,44677,Black Drawstring Trackpants,Athleisure | Sweatpants,1699.0


In [ ]:
product_id = 416595
recommended_products = get_recommendations(product_id, merged_df, top_n=10)
recommended_products[['product_id','name','Category Group','price']]

,product_id,name,Category Group,price
41,403462,Dark Blue Low Rise Ben Skinny Jeans,Jeans | Skinny Fit | 12226565 | Fashion,3999.0
36,403461,Dark Blue Low Rise Washed Ben Skinny Fit Jeans,Jeans | Skinny Fit | 12226563 | Fashion,3999.0
27,400873,Blue Low Rise Distressed Ben Skinny Fit Jeans,Jeans | Skinny Fit,4499.0
12,409480,Ecru Low Rise Liam Skinny Fit Jeans,Jeans | Skinny Fit | 12257861 | FASHION NOOS,3299.0
9,391784,Light Blue Low Rise Liam Skinny Jeans,Jeans | Skinny Fit,4499.0
43,414395,Grey Low Rise Ben Skinny Fit Jeans,Jeans | Skinny Fit | 12269206 | SMU,2799.0
22,403504,Blue Low Rise Ben Skinny Jeans,Jeans | Skinny Fit | 12226856 | Fashion,3499.0
31,407698,Blue Low Rise Ben Skinny Jeans,Jeans | Skinny Fit | 12244470 | Fashion,4499.0
23,400839,Blue Low Rise Ben Skinny Fit Jeans,Jeans | Skinny Fit,4999.0
42,403467,Dark Blue Low Rise Ben Skinny Jeans,Jeans | Skinny Fit | 12226591 | Fashion,4499.0


In [ ]:
value_counts = merged_df['mpn'].value_counts()

filtered_groups = value_counts[value_counts < 50].index

filtered_groups

Index(['Masks', 'Beanie', 'Pyjamas', 'Trousers', 'Wallets', 'Trackpants',
       'Sweaters', 'Sliders', 'Fashion Vests', 'Boots', 'Scarves', 'Sleepwear',
       'Bags', 'Cardigans', 'Flip Flops', 'Joggers', 'Sandals', 'Gloves',
       'Swimshorts', 'Vests', 'Slip Ons', 'Chinos', 'Fragrances', 'Hats', '',
       'Card Holder', 'Night Suit Set', 'Tailored Trousers', 'Cargo Pants',
       'Sunglasses', 'Tracksuits', 'Sleepwear Set', 'Card Case', 'Waistcoats',
       'Laptop Sleeves'],
      dtype='object', name='mpn')

In [ ]:
value_counts

mpn
T-Shirts             1950
Shirts               1340
Jeans                 842
Sweatshirts           298
Jackets               244
Sneakers              232
Briefs                182
Trunks                178
Caps                  165
Shorts                162
Sweatpants            118
Pants                 116
Socks                 107
Belts                 107
Pullovers              91
Boxers                 82
Blazers                60
Sweatshorts            54
Backpacks              52
Masks                  48
Beanie                 39
Pyjamas                38
Trousers               31
Wallets                30
Trackpants             30
Sweaters               26
Sliders                25
Fashion Vests          22
Boots                  21
Scarves                20
Sleepwear              16
Bags                   11
Cardigans              11
Flip Flops             10
Joggers                10
Sandals                 8
Gloves                  7
Swimshorts              6
Vests   

In [ ]:
product_id = 60589
recommended_products = get_recommendations(product_id, merged_df, top_n=10)
recommended_products[['product_id','name','Category Group','price']]

,product_id,name,Category Group,price
4,379773,Pack of 3 Blue All Over Print B95 3 PLY Mask,Masks,599.0
20,60368,Pack of 3 Logo Print Knit 3PLY Mask,Masks,699.0
7,60461,Pack of 2 Blue Lightweight Denim 3PLY Mask,Masks,499.0
27,60369,Pack of 3 Black Text Print Knit 3PLY Mask,Masks,699.0
19,378751,Pack of 2 Grey 3PLY Mask with Exhale Valve,Masks,799.0
1,60462,Pack of 3 Self-Design Ligthweight Denim 3PLY Mask,Masks,699.0
43,379772,Grey Abstract Print Mask with 3 Changeable PM ...,Masks,899.0
33,379039,Pack of 3 Black Graphic Print Knit 3PLY Mask,Masks,699.0
39,379304,Pack of 5 White Logo Print N95 Mask with PM2.5...,Masks,799.0
5,379792,Pack of 3 All Over Print B95 3PLY Mask,Masks,599.0


In [ ]:
product_id = 56738
recommended_products = get_recommendations(product_id, merged_df, top_n=10)
recommended_products[['product_id','name','Category Group','price']]

,product_id,name,Category Group,price
11,404886,Pack Of 3 Terry Ankle Length Socks,Accessories | Socks | 12243243 | Fashion,599.0
48,404876,Pack of 3 White Terry Ankle Length Socks,Accessories | Socks | 12243233 | Fashion,599.0
44,404877,Pack Of 3 Terry Ankle Length Socks,Accessories | Socks | 12243234 | Fashion,599.0
47,404843,Pack of 3 Polka Dot No-Show Socks,Accessories | Socks | 12234391 | Fashion,549.0
45,404872,Pack Of 3 Blue Terry Mid Length Socks,Accessories | Socks | 12243224 | Fashion,679.0
35,404883,Pack Of 3 Terry Mid Length Socks,Accessories | Socks | 12243240 | Fashion,679.0
23,412708,Pack of 2 Fruit Print Mid-Length Socks,Accessories | Socks | 12264879 | FASHION,599.0
28,383911,Pack of 3 Footies - Multi-coloured,Accessories | Socks,549.0
9,412720,Pack of 3 Racer Print No-Show Socks,Accessories | Socks | 12264870 | FASHION,499.0
1,412714,Pack of 3 Logo Print No-Show Socks,Accessories | Socks | 12264868 | FASHION,499.0


In [ ]:
product_id = 416595
recommended_products = get_recommendations(product_id, merged_df, top_n=10)
recommended_products[['product_id','name','Category Group','price']]

,product_id,name,Category Group,price
41,403462,Dark Blue Low Rise Ben Skinny Jeans,Jeans | Skinny Fit | 12226565 | Fashion,3999.0
36,403461,Dark Blue Low Rise Washed Ben Skinny Fit Jeans,Jeans | Skinny Fit | 12226563 | Fashion,3999.0
27,400873,Blue Low Rise Distressed Ben Skinny Fit Jeans,Jeans | Skinny Fit,4499.0
12,409480,Ecru Low Rise Liam Skinny Fit Jeans,Jeans | Skinny Fit | 12257861 | FASHION NOOS,3299.0
9,391784,Light Blue Low Rise Liam Skinny Jeans,Jeans | Skinny Fit,4499.0
43,414395,Grey Low Rise Ben Skinny Fit Jeans,Jeans | Skinny Fit | 12269206 | SMU,2799.0
22,403504,Blue Low Rise Ben Skinny Jeans,Jeans | Skinny Fit | 12226856 | Fashion,3499.0
31,407698,Blue Low Rise Ben Skinny Jeans,Jeans | Skinny Fit | 12244470 | Fashion,4499.0
23,400839,Blue Low Rise Ben Skinny Fit Jeans,Jeans | Skinny Fit,4999.0
42,403467,Dark Blue Low Rise Ben Skinny Jeans,Jeans | Skinny Fit | 12226591 | Fashion,4499.0


In [ ]:
product_id = 400380
recommended_products = get_recommendations(product_id, merged_df, top_n=10)
recommended_products[['product_id','name','Category Group','price']]

,product_id,name,Category Group,price
7,398025,Dark Blue Structured Blazer,Tailoring | Blazers,5999.0
48,402167,Navy Blue Blazer,Tailoring | Blazers,7999.0
9,397080,Maroon Striped Blazer,Tailoring | Blazers,8999.0
20,408404,Black Double Button Twill Blazer,Tailoring | Blazers | 12241993 | FASHION,5999.0
1,400378,Navy Blue Tailored Blazer,Tailoring | Blazers,8499.0
35,399315,Blue Denim Blazer,Tailoring | Blazers,9999.0
37,401703,White Striped Double Button Blazer,Tailoring | Blazers,8999.0
16,388444,Light Blue Tailored Formal Blazer,Tailoring | Blazers,6999.0
25,401001,Pink Knit Blazer,Tailoring | Blazers,6999.0
22,407657,Brown Knitted Slim Fit Blazer,Tailoring | Blazers | 12246368 | Fashion,7999.0


In [ ]:
product_id = 398158
recommended_products = get_recommendations(product_id, merged_df, top_n=10)
recommended_products[['product_id','name','Category Group','price']]

,product_id,name,Category Group,price
14,392503,Brown Leather Belt,Accessories | Belts,1999.0
36,403651,Black Leather Belt,Accessories | Belts | 12243717 | FASHION,1999.0
21,400070,Black Leather Belt,Accessories | Belts,1999.0
22,404607,Brown Leather Belt,Accessories | Belts | 12243725 | Fashion,1699.0
29,403339,Brown Printed Leather Belt,Accessories | Belts | 12243719 | Fashion,1699.0
13,403653,Brown Leather Belt,Accessories | Belts | 12243727 | FASHION,1999.0
23,392511,Brown Leather Belt,Accessories | Belts,1699.0
25,414315,Tan Leather Belt,Accessories | Belts | 12268297 | FASHION,1499.0
5,414306,Brown Leather Textured Belt,Accessories | Belts | 12268285 | FASHION,1999.0
16,414305,Brown Leather Belt,Accessories | Belts | 12268286 | FASHION,1999.0


In [ ]:
merged_df.loc[merged_df['mpn']=='Laptop Sleeves']

,product_id,description,g:product_type,name,jan,mpn,price,special_price,Category Group,Specifications,combined_text_features,volume_of_sales,discount_price_ratio,price_sales_ratio,Quarter1_sales,Quarter2_sales,Quarter3_sales,Quarter4_sales
1906,399094,Black Corduroy Laptop Sleeve,Men &gt; Accessories &gt; Bags,Black Corduroy Laptop Sleeve,AW22 FA,Laptop Sleeves,1299.0,1039.0,Accessories,Black | 100% Cotton | Plain Coloured | Casual ...,Black Corduroy Laptop Sleeve Black Corduroy La...,84.0,0.799846,15.464286,0.0,1.0,0.0,0.0


In [ ]:
product_id = 399094
recommended_products = get_recommendations(product_id, merged_df, top_n=10)
recommended_products[['product_id','name','Category Group','price']]

,product_id,name,Category Group,price
3,397907,Black Multi-Purpose Duffle Bag,Accessories | Bags,3999.0
47,409098,BLACK #03 EAU DE TOILETTE FRAGRANCE - 75ML,Accessories | Fragrances | 12163325 | FASHION,2499.0
24,400069,Brown Colourblocked Backpack,Accessories | Bags,2999.0
41,410294,Red Colourblocked Chestpack,Accessories | Backpacks | 12257507 | FASHION,1699.0
13,388047,Grey Text Print Backpack,Accessories | Bags,2999.0
12,410293,Black Colourblocked Chestpack,Accessories | Backpacks | 12257507 | FASHION,1699.0
37,410699,Blue Colourblocked Backpack,Accessories | Bags | 12257741 | FASHION NOOS,2999.0
42,399093,Grey Chestpack,Accessories | Bags,1499.0
28,413346,Grey Backpack,Accessories | Backpacks | 12267692 | NOOS,2999.0
33,414971,Dark Brown Aztec Print Backpack,Accessories | Bags | 12267704 | FASHION,3999.0


In [ ]:
merged_df.loc[merged_df['product_id']==399094]

,product_id,description,g:product_type,name,jan,mpn,price,special_price,Category Group,Specifications,combined_text_features,volume_of_sales,discount_price_ratio,price_sales_ratio,Quarter1_sales,Quarter2_sales,Quarter3_sales,Quarter4_sales
1906,399094,Black Corduroy Laptop Sleeve,Men &gt; Accessories &gt; Bags,Black Corduroy Laptop Sleeve,AW22 FA,Laptop Sleeves,1299.0,1039.0,Accessories,Black | 100% Cotton | Plain Coloured | Casual ...,Black Corduroy Laptop Sleeve Black Corduroy La...,84.0,0.799846,15.464286,0.0,1.0,0.0,0.0


In [3]:
# below code is to create recommendations for all products 

In [ ]:
# Function to create the dataset with recommendations for all products
def create_recommendation_dataset(merged_df, top_n=10):
    # Initialize lists to store the data
    product_ids = []
    product_names = []
    categories = []
    prices = []
    recommended_product_ids_list = []
    recommended_categories_list = []
    recommended_product_name_list = []
    average_recommended_prices = []

    # Iterate over each product in the DataFrame
    for idx, row in merged_df.iterrows():
        product_id = row['product_id']
        product_name = row["name"]
        category = row['Category Group']
        price = row['price']

        # Get recommendations for the current product_id
        recommended_products = get_recommendations(product_id, merged_df, top_n)

        # Extract the required information from the recommendations
        recommended_product_ids = recommended_products['product_id'].tolist()
        recommended_categories = recommended_products['Category Group'].tolist()
        recommended_product_name = recommended_products["name"].tolist()
        average_price = recommended_products['price'].mean()

        # Append the data to the lists
        product_ids.append(product_id)
        product_names.append(product_name)
        categories.append(category)
        prices.append(price)
        recommended_product_ids_list.append(recommended_product_ids)
        recommended_categories_list.append(recommended_categories)
        recommended_product_name_list.append(recommended_product_name)
        average_recommended_prices.append(average_price)

    # Create a new DataFrame with the collected data
    recommendation_df = pd.DataFrame({
        'product_id': product_ids,
        'product_name': product_names,
        'category': categories,
        'price': prices,
        'recommended_product_ids': recommended_product_ids_list,
        'recommended_product_name': recommended_product_name_list,
        'recommended_categories': recommended_categories_list,
        'avg_price_recommended': average_recommended_prices
    })

    return recommendation_df

# Create the recommendation dataset
recommendation_dataset = create_recommendation_dataset(merged_df, top_n=10)

# Display the first few rows of the recommendation dataset
print(recommendation_dataset.head())


   product_id                          product_name                 category  \
0       58641               Olive Crew Neck T-shirt     T-Shirts | Crew Neck   
1      383422  Black All Over Caution Print Pyjamas     Loungewear | Pyjamas   
2       59108        Black Contrast Detail Sneakers      Footwear | Sneakers   
3       44149           Black Drawstring Trackpants  Athleisure | Sweatpants   
4       44677           Black Drawstring Trackpants  Athleisure | Sweatpants   

    price                            recommended_product_ids  \
0   799.0  [397205, 397206, 398244, 394911, 58223, 410919...   
1  1299.0  [401156, 394852, 401158, 401162, 394265, 40116...   
2  5999.0  [399107, 399108, 405308, 406523, 405314, 40674...   
3  1599.0  [44150, 389321, 385918, 385917, 395463, 386222...   
4  1699.0  [44150, 44151, 389321, 385918, 385917, 395463,...   

                            recommended_product_name  \
0  [White Crew Neck T-shirt, Black Crew Neck T-sh...   
1  [Red Printed Pyjama

In [ ]:
recommendation_dataset

,product_id,product_name,category,price,recommended_product_ids,recommended_product_name,recommended_categories,avg_price_recommended
0,58641,Olive Crew Neck T-shirt,T-Shirts | Crew Neck,799.0,"[397205, 397206, 398244, 394911, 58223, 410919...","[White Crew Neck T-shirt, Black Crew Neck T-sh...","[T-Shirts | Crew Neck, T-Shirts | Crew Neck, T...",1509.0
1,383422,Black All Over Caution Print Pyjamas,Loungewear | Pyjamas,1299.0,"[401156, 394852, 401158, 401162, 394265, 40116...","[Red Printed Pyjamas, Blue Printed Pyjamas, Wh...","[Loungewear | Pyjamas, Loungewear | Pyjamas, L...",1279.0
2,59108,Black Contrast Detail Sneakers,Footwear | Sneakers,5999.0,"[399107, 399108, 405308, 406523, 405314, 40674...","[Black Hi-Top Sneakers, White Colourblocked Hi...","[Footwear | Sneakers, Footwear | Sneakers, Foo...",4569.0
3,44149,Black Drawstring Trackpants,Athleisure | Sweatpants,1599.0,"[44150, 389321, 385918, 385917, 395463, 386222...","[Dark Grey Drawstring Trackpants, Blue Mid Ris...","[Athleisure | Sweatpants, Athleisure | Sweatpa...",1829.0
4,44677,Black Drawstring Trackpants,Athleisure | Sweatpants,1699.0,"[44150, 44151, 389321, 385918, 385917, 395463,...","[Dark Grey Drawstring Trackpants, Navy Drawstr...","[Athleisure | Sweatpants, Athleisure | Sweatpa...",1789.0
...,...,...,...,...,...,...,...,...
6832,416595,Light Blue Low Rise Ben Skinny Fit Jeans,Jeans | Skinny Fit | 12271267 | SMU,2999.0,"[403461, 400873, 391784, 409480, 414395, 40350...",[Dark Blue Low Rise Washed Ben Skinny Fit Jean...,"[Jeans | Skinny Fit | 12226563 | Fashion, Jean...",4039.0
6833,416811,,,0.0,"[414184, 413538, 415886, 415258, 415261, 41418...","[Boys Blue Smiley Print Co-ord Set Shorts, Boy...","[Bottomwear | Shorts | 9003883 | FASHION, Bott...",1469.0
6834,416812,,,0.0,"[416511, 416512, 416498, 414693, 414684, 41467...","[Looney Tunes Blue Bugs Bunny Print T-shirt, L...","[T-Shirts | Crew Neck | 12271017 | FASHION, T-...",1449.0
6835,416813,,,0.0,"[416512, 416513, 416498, 414693, 415863, 41586...",[Looney Tunes White Bugs Bunny Colourblocked T...,"[T-Shirts | Crew Neck | 12271021 | FASHION, T-...",1489.0


In [ ]:
recommendation_dataset.to_csv("recommendation_item_item.csv",index=False)